In [2]:
import numpy
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from importlib import reload
import InputList
import BinaryReader
import Preprocessor
import models
import Callbacks

2022-02-10 09:56:41.513619: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-10 09:56:43.014690: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-02-10 09:56:43.014706: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-02-10 09:56:43.015374: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-10 09:56:43.015401: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [59]:
reload(Callbacks)

2022-02-10 15:03:01.800516: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-02-10 15:03:01.800579: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.


<module 'Callbacks' from '/home/julius/dataspellprojects/oct-classifier/Callbacks.py'>

2022-02-10 15:03:01.800666: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-02-10 15:03:01.801004: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-02-10 15:03:01.801033: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-02-10 15:03:01.801085: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


## Make this such that it can be used for prediction and training pipeline

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
reader = BinaryReader.BinaryReader()  # TODO: Normalizer
training_dataset, validation_dataset = reader.create_training_datasets(InputList.training_files)
preprocesser = Preprocessor.Preprocessor(training_dataset)

Num GPUs Available:  0
[('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/H6/rechts/raw_1536x2048x2045x2_8042.bin', 0), ('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D85/rechts/raw_1536x2048x2045x2_19200.bin', 1)]


<PrefetchDataset shapes: ((1536, 23, 28, 1), ()), types: (tf.uint16, tf.uint8)>

In [51]:
print("Creating Normalization Layer:")
normalization_layer = preprocesser.normalize_layer()

Creating Normalization Layer:


In [8]:
#normalization_weights = normalization_layer.get_weights()
#print("Before:" + str(normalization_weights))
normalization_weights_path = "results/normalization.npy"
#np.save(normalization_weights_path, normalization_weights)
normalization_layer.set_weights(np.load(normalization_weights_path))
print("After" + str(normalization_layer.get_weights()))

After[32762.3, 4103221.0, 9891840]


In [52]:
normalization_layer.set_weights(np.array([32743.002, 5470130.0, 25706913792]))

In [53]:
model = models.classiRaw3D(training_dataset.element_spec[0].shape, normalization_layer, reconstruction=False)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1536, 23, 28, 1)] 0         
_________________________________________________________________
normalization (Normalization (None, 1536, 23, 28, 1)   3         
_________________________________________________________________
conv3d (Conv3D)              (None, 1536, 23, 28, 64)  1792      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 768, 11, 14, 64)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 768, 11, 14, 128)  221312    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 384, 5, 7, 128)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 384, 5, 7, 192)    663744

In [66]:
history = model.fit(
    reader.create_test_dataset([("/mnt/NewHDD/train_data/raw_1536x2048x2045x2_9201.bin", 0)]).batch(4).take(3),
    epochs=5,
    #validation_data=Preprocessor.Preprocessor(validation_dataset).batch(5).take(4),
    callbacks=Callbacks.tboard_callback
)
#model.save('savedModels/first')

Epoch 1/5


2022-02-10 17:10:56.773221: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-02-10 17:10:56.773239: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.


      2/Unknown - 63s 31s/step - loss: 3.7162

2022-02-10 17:11:59.289101: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-02-10 17:11:59.290457: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-02-10 17:11:59.291988: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/new/train/plugins/profile/2022_02_10_17_11_59
2022-02-10 17:11:59.293364: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/new/train/plugins/profile/2022_02_10_17_11_59/zeissstudent1.trace.json.gz
2022-02-10 17:11:59.294435: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/new/train/plugins/profile/2022_02_10_17_11_59
2022-02-10 17:11:59.294473: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for memory_profile.json.gz to logs/new/train/plugins/profile/2022_02_10_17_11_59/zeissstudent1.memory_profile.json.gz
2022-02-10 17

3/3 [==============================] - 94s 31s/step - loss: 3.6748
Epoch 2/5
3/3 [==============================] - 94s 31s/step - loss: 3.4319
Epoch 3/5
3/3 [==============================] - 94s 31s/step - loss: 3.2000
Epoch 4/5
3/3 [==============================] - 94s 31s/step - loss: 2.9790
Epoch 5/5
3/3 [==============================] - 94s 31s/step - loss: 2.7695


In [69]:
reader.create_test_dataset([("/mnt/NewHDD/train_data/raw_1536x2048x2045x2_9201.bin", 0)])

In [3]:
new_model = tf.keras.models.load_model('savedModels/first')

2022-02-10 09:56:49.761062: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-10 09:56:49.761259: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-10 09:56:49.761278: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (zeissstudent1): /proc/driver/nvidia/version does not exist
2022-02-10 09:56:49.761470: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-10 09:56:49.761934: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [82]:
test_dataset = reader.create_test_dataset([InputList.healthy_training_files[0]])
output_healthy = new_model.predict(test_dataset.batch(20))

In [74]:
test_dataset = reader.create_test_dataset([InputList.diabetic_training_files[4]])
output = new_model.predict(test_dataset.batch(20))

In [85]:
np.max(output_healthy)

-4.4209824

In [37]:
test_dataset = reader.create_test_dataset([InputList.healthy_training_files[4]])
output_healthy_val = new_model.predict(test_dataset.batch(20), callbacks=Callbacks.tboard_callback)

In [39]:
test_dataset = reader.create_test_dataset([("/mnt/NewHDD/train_data/raw_1536x2048x2045x2_9201.bin", 0)])
output_some_file = new_model.predict(test_dataset.batch(20), callbacks=Callbacks.tboard_callback)

KeyboardInterrupt: 

In [28]:
test_dataset = reader.create_test_dataset([InputList.healthy_testing_files[0]])
output_healthy_test = new_model.predict(test_dataset.batch(20))

In [6]:
test_dataset = reader.create_test_dataset([InputList.diabetic_testing_files[0]])
output_diabetic_test = new_model.predict(test_dataset.batch(20), callbacks=)

2022-02-10 09:57:10.869736: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-10 09:57:10.886408: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2899885000 Hz
2022-02-10 09:57:12.287878: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5064622080 exceeds 10% of free system memory.
2022-02-10 09:57:17.295475: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5064622080 exceeds 10% of free system memory.
2022-02-10 09:57:22.651924: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5064622080 exceeds 10% of free system memory.
2022-02-10 09:57:28.308051: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5064622080 exceeds 10% of free system memory.
2022-02-10 09:57:33.802789: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5064622080 exceeds 10% of free system memory.


In [9]:
with open("instance_test/output_buffer.npy", "wb") as f:
    np.save(f, output_diabetic_test)

In [10]:
with open("instance_test/output_buffer.npy", "rb") as f:
    buffi = np.load(f)

In [11]:
buffi

array([[6.119612 ],
       [6.305094 ],
       [7.2212224],
       ...,
       [6.104448 ],
       [6.5245843],
       [6.7683864]], dtype=float32)

In [48]:
import Visualization
reload(Visualization)
testing_diabetic_image = Visualization.ImageVisualizer(output_diabetic_test,
                                                       info_map=None,
                                                       background_image_path=
                                                       "/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D71/rechts/retina_1536x2048x2045x2_2375.png")
testing_diabetic_image.plot_results_map("diabetic_test")

In [49]:
testing_healthy_image = Visualization.ImageVisualizer(output_healthy_test,
                                                       info_map=None,
                                                       background_image_path=
                                                       "/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D71/rechts/retina_1536x2048x2045x2_2375.png")
testing_healthy_image.plot_results_map("healthy_test")

In [ ]:
output_healthy_val

In [ ]:
i = 1

In [1]:
np.argmax(output_diabetic_test)

NameError: name 'np' is not defined

In [ ]:
reload(models)
model = models.classiRaw3D(training_dataset.element_spec[0].shape, normalization_layer, reconstruction=False)

In [224]:
training_dataset

<PrefetchDataset shapes: ((1536, 23, 28, 1), ()), types: (tf.uint16, tf.uint8)>

In [8]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)

Num GPUs Available:  0


In [18]:
model.load_weights("checkpoints/best_model")

In [12]:
with open("checkpoints/best_model.data-00000-of-00001") as testi:
    print(testi)

<_io.TextIOWrapper name='checkpoints/best_model.data-00000-of-00001' mode='r' encoding='UTF-8'>


print_function(Bag Number, dataset)

-> In einem Bag sind 7000 Instanzen

# Bag Level Model:

Ich hab ganz viele Softmaxes und von denen nehm ich einfach den Max Wert

In [68]:
test_dataset = reader.create_test_dataset([InputList.diabetic_training_files[4]])
output = model.predict(test_dataset.batch(20))

2022-02-08 18:00:53.002220: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5064622080 exceeds 10% of free system memory.
2022-02-08 18:00:57.834422: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5064622080 exceeds 10% of free system memory.
2022-02-08 18:01:03.134897: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5064622080 exceeds 10% of free system memory.
2022-02-08 18:01:08.671641: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5064622080 exceeds 10% of free system memory.
2022-02-08 18:01:14.168709: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5064622080 exceeds 10% of free system memory.
